In [1]:
import nbimporter
import pandas as pd
from collections import OrderedDict
import read_report_cobol as cobol_reader # type: ignore
import read_report_java as java_reader # type: ignore

def get_employee_types():
    employee_types = ['clt', 'autarquico']
    return employee_types

def get_common_table_types():
    table_types = ['apropriado', 'realizado']
    return table_types

def subtract_dataframes(df1: pd.DataFrame, df2: pd.DataFrame, items:list) -> pd.DataFrame:
    df3 = df1 - df2
    if items is not None:
        df3 = pd.concat([items, df3], axis=1)
    return df3

def correct_autarquico(cobol:pd.DataFrame) -> pd.DataFrame:
    correct_cobol = cobol.iloc[:3,:].copy()
    total_line = pd.to_numeric(correct_cobol.iloc[0, 1:], errors='coerce') + pd.to_numeric(correct_cobol.iloc[2, 1:], errors='coerce')
    correct_cobol.iloc[1, 1:4] = 0.00
    correct_cobol.iloc[1, 5] = correct_cobol.iloc[1, 4]
    correct_cobol.loc[3] = ['total'] + total_line.tolist()
    correct_cobol = correct_cobol.reindex([0,2,3,1])
    correct_cobol = correct_cobol.reset_index(drop=True)
    return correct_cobol

def compare_clt_apropriado(cobol:pd.DataFrame, java:pd.DataFrame) -> pd.DataFrame:
    items = java.iloc[:,0]
    df_j =  java.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
    df_c = cobol.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
    return subtract_dataframes(df_j, df_c, items), cobol, java

def compare_clt_realizado(cobol:pd.DataFrame, java:pd.DataFrame) -> pd.DataFrame:
    items = cobol.iloc[:,0]
    df_j  = java.iloc[2:,1:].apply(pd.to_numeric, errors='coerce')
    df_j  = df_j.reset_index(drop=True)
    df_c  = cobol.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
    return subtract_dataframes(df_j, df_c, items), cobol, java

def compare_autarquico(cobol:pd.DataFrame, java:pd.DataFrame) -> pd.DataFrame:
    cobol = correct_autarquico(cobol)
    return compare_clt_apropriado(cobol, java)

cobol_file_path = "RHPP05LA.PDF"
java_file_path  = "RHPP05LA - GERAL.pdf"

results = {}

# lê os arquivos, adiciona a um dicionario e ordena as chaves
cobol_report = cobol_reader.get_cobol_data_from(cobol_file_path)
java_report = java_reader.get_java_data_from(java_file_path)
cobol_report = OrderedDict(sorted(cobol_report.items()))
java_report  = OrderedDict(sorted(java_report.items()))

# verifica se as chaves sao iguais e na mesma ordem
if list(cobol_report.keys()) == list(java_report.keys()):
    for (c_key, c_item), (j_key, j_item) in zip(cobol_report.items(), java_report.items()):
        for emp_type in get_employee_types():
            for tb_type in get_common_table_types():
                if emp_type == get_employee_types()[0] and tb_type == get_common_table_types()[0]:
                    df_diff = compare_clt_apropriado(c_item[emp_type][tb_type], j_item[emp_type][tb_type])
                elif emp_type == get_employee_types()[0] and tb_type == get_common_table_types()[1]:
                    df_diff = compare_clt_realizado(c_item[emp_type][tb_type], j_item[emp_type][tb_type])
                elif emp_type == get_employee_types()[1]:
                    df_diff = compare_autarquico(c_item[emp_type][tb_type], j_item[emp_type][tb_type])
                else:
                    print("Tabela não identificada")
else:
    print("Chaves diferentes entre os arquivos! Validar PDF.")

abr 16, 2024 6:25:21 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font Symbol
abr 16, 2024 6:25:21 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font ZapfDingbats


'diferenca: (java) - (cobol)'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,3115.52,1038.13,4153.65,3115.52,7269.17
1,spprevcom,0.00,0.00,0.00,43.59,43.59
2,total,3115.52,1038.13,4153.65,3159.11,7312.76
3,spprev,0.00,0.00,0.00,10407.11,10407.11


'------------'

'diferenca: (java) - (cobol)'

...

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,-1.59,-1.51,-3.1,-1.59,-4.69
1,spprevcom,0.00,0.00,0.0,31.96,31.96
2,total,-1.59,-1.51,-3.1,30.37,27.27
3,spprev,0.00,0.00,0.0,24885.71,24885.71


'------------'

'diferenca: (java) - (cobol)'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,272527.7,95445.84,367973.54,0.0,367973.54
1,spprevcom,0.0,0.00,0.00,0.0,0.00
2,total,272527.7,95445.84,367973.54,0.0,367973.54
3,spprev,0.0,0.00,0.00,0.0,0.00


'------------'

In [2]:
code = 40434

# display("cobol", "clt")
display("apropriado", cobol_report[code]['clt']['apropriado'])
# display("realizado", cobol[code]['clt']['realizado'])
# display("autarquico")
# display("apropriado", cobol[code]['autarquico']['apropriado'])
# display("realizado", cobol[code]['autarquico']['realizado'])
# display("java", "clt")
display("apropriado", java_report[code]['clt']['apropriado'])
# display("realizado", java[code]['clt']['realizado'])
# display("autarquico")
# display("apropriado", java[code]['autarquico']['apropriado'])
# display("realizado", java[code]['autarquico']['realizado'])

items = java_report[code]['clt']['apropriado'].iloc[:,0]
df1 = java_report[code]['clt']['apropriado'].iloc[:,1:].apply(pd.to_numeric, errors='coerce')
df2 = cobol_report[code]['clt']['apropriado'].iloc[:,1:].apply(pd.to_numeric, errors='coerce')

df3 = df1 - df2
df3 = pd.concat([items, df3], axis=1)

display(df3)

'apropriado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,360940.90,120313.76,481254.66,360940.90,842195.56
1,inss,61317.13,20439.04,81756.17,61317.13,143073.30
2,ac_trab,1531.42,510.10,2041.52,1531.42,3572.94
3,fgts,24526.11,8175.33,32701.44,24526.11,57227.55
4,spprevcom,0.00,0.00,0.00,1668.03,1668.03
5,total_encargos,87374.66,29124.47,116499.13,89042.69,205541.82
6,total,448315.56,149438.23,597753.79,449983.59,1047737.38


'apropriado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,364641.26,121545.68,486186.94,364641.26,850828.20
1,inss,72926.76,24308.00,97234.76,62057.50,159292.26
2,ac_trab,1821.50,605.96,2427.46,1549.83,3977.29
3,fgts,29169.32,9721.89,38891.21,24822.20,63713.41
4,spprevcom,0.00,0.00,0.00,1959.04,1959.04
5,total_encargos,103917.58,34635.85,138553.43,90388.57,228942.00
6,total,468558.84,156181.53,624740.37,455029.83,1079770.20


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,3700.36,1231.92,4932.28,3700.36,8632.64
1,inss,11609.63,3868.96,15478.59,740.37,16218.96
2,ac_trab,290.08,95.86,385.94,18.41,404.35
3,fgts,4643.21,1546.56,6189.77,296.09,6485.86
4,spprevcom,0.00,0.00,0.00,291.01,291.01
5,total_encargos,16542.92,5511.38,22054.30,1345.88,23400.18
6,total,20243.28,6743.30,26986.58,5046.24,32032.82
